In [6]:
import json
import networkx as nx
import numpy as np

import functions

In [7]:
#function to read .json file of the input graph
def readPreferenceGraph(filePath) :
    with open(filePath) as file :
        data = json.load(file)

    G = nx.MultiDiGraph()

    for edge in data["edges"] :
        G.add_edge(edge["source"], edge["target"], opinion=edge["opinion"], questionItem=edge["question item"])
    
    return G

In [8]:
def createInteractionGraph(preferenceGraph, threshold, maxInteractions, maxTimestamp) :
    n = preferenceGraph.number_of_nodes()
    activitiesCount = 6

    interactionGraph = []
    interactionGraphIntensity = []
    
    degrees = [0] * n
   
        
    # list of possible peers to inteact for each person
    possiblePeers = functions.analyzeQuiz(prefGraph=preferenceGraph, \
                                          threshold=threshold, \
                                          numNodes=n)

    # ------------------------------------- #
    #    interactions at timetsamp t = 0    #
    # ------------------------------------- #
    # 1st step: interactions based on a scale-free network model with m=3
    interactionGraph.append(nx.barabasi_albert_graph(n=preferenceGraph.number_of_nodes(), \
                                                     m=3))

    # 2nd step: add random activities as weights
    for u, v in interactionGraph[0].edges() :
        interactionGraph[0][u][v]["interaction_index"] = np.random.choice(range(activitiesCount))
    
    # 3rd step: calculate intensity
    snapshotIntensity = functions.calculateIntensityForSnapshot(snapshotGraph=interactionGraph[0], 
                                                                numNodes=n)
    interactionGraphIntensity.append(snapshotIntensity)

    # 4th step: add snapshot degrees to the total number of degrees
    for node in range(n) :
        degrees[node] += interactionGraph[0].degree(node) 
    

    # -------------------------------------- #
    #    interactions at timetsamp t >= 1    #
    # -------------------------------------+ #
    for time in range(1, maxTimestamp + 1) :
        
        # 1st step: create new snapshot
        # 2nd step: add activities as weights
        snapshot = functions.createSnapshot(numNodes=n, \
                                            maxInteractions=maxInteractions, \
                                            degrees=degrees, \
                                            possiblePeers=possiblePeers, \
                                            activitiesCount=activitiesCount)
        interactionGraph.append(snapshot)
        
        # 3rd step: calculate intensity
        snapshotIntensity = functions.calculateIntensityForSnapshot(snapshotGraph=interactionGraph[time], \
                                                                    numNodes=n)
        interactionGraphIntensity.append(snapshotIntensity)
        
        # 4th step: add snapshot degrees to the total number of degrees
        for node in range(n) :
            degrees[node] += interactionGraph[time].degree(node) 

    
    # calculate indexes for each snapshot
    graphSequenceIndexes = []
    for time in range(maxTimestamp + 1) :
        snapshotIndexes = functions.calculateIndexes(prefGraph=preferenceGraph, \
                                                     snapshotGraph=interactionGraph[time], \
                                                     numNodes=n)
        graphSequenceIndexes.append(snapshotIndexes)


    return {
        "graph" : interactionGraph,
        "sequence intensity": interactionGraphIntensity,
        "individual": {
            "popularity": np.average([item["individual"]["popularity"] for item in graphSequenceIndexes]),
            "antipathy": np.average([item["individual"]["antipathy"] for item in graphSequenceIndexes]),
            "affective connection": np.average([item["individual"]["affective connection"] for item in graphSequenceIndexes]),
            "sociometric status": np.average([item["individual"]["sociometric status"] for item in graphSequenceIndexes]),
            "positive expansion": np.average([item["individual"]["positive expansion"] for item in graphSequenceIndexes]),
            "negative expansion": np.average([item["individual"]["negative expansion"] for item in graphSequenceIndexes]),
            "realistic perception": np.average([item["individual"]["realistic perception"] for item in graphSequenceIndexes])
        },
        "group": {
            "association": np.average([item["group"]["association"] for item in graphSequenceIndexes]),
            "dissociation": np.average([item["group"]["dissociation"] for item in graphSequenceIndexes]),
            "cohesion": np.average([item["group"]["cohesion"] for item in graphSequenceIndexes]),
            "social intensity": np.average([item["group"]["social intensity"] for item in graphSequenceIndexes]),
        }
    }

In [9]:
path = "./preferenceGraph.json"

options = {
    "threshold" : 0.4,
    "maxInteractions" : 200,
    "maxTimestamp" : 50
}

G = createInteractionGraph(preferenceGraph=readPreferenceGraph(path), **options)

print(f"Graph Sequence Intensity: {G['sequence intensity']}")
print("\nSnapshots added...")
for i in range(6) :
    print(f"at t = {i}: {G['graph'][i]}")

print("\n")
print(G["individual"])
print(G["group"])

Graph Sequence Intensity: [13244, 4177, 9364, 9278, 4330, 9532, 4403, 9292, 4247, 4398, 9185, 9403, 4281, 4186, 9146, 4442, 9478, 4248, 4503, 4595, 9041, 9360, 9219, 9068, 4430, 9215, 4440, 9625, 9679, 8941, 4055, 9246, 4199, 4340, 4212, 9192, 8201, 3076, 2411, 5091, 1777, 1376, 2494, 1383, 546, 810, 228, 276, 219, 128, 42]

Snapshots added...
at t = 0: Graph with 100 nodes and 291 edges
at t = 1: Graph with 100 nodes and 196 edges
at t = 2: Graph with 100 nodes and 293 edges
at t = 3: Graph with 100 nodes and 293 edges
at t = 4: Graph with 100 nodes and 196 edges
at t = 5: Graph with 100 nodes and 295 edges


{'popularity': 0.027207367795603087, 'antipathy': 0.013628441275500098, 'affective connection': 0.23860882118970356, 'sociometric status': 0.018484848484848482, 'positive expansion': 0.027207367795603087, 'negative expansion': 0.013628441275500098, 'realistic perception': 0.21863780978964917}
{'association': 0.007704495939790058, 'dissociation': 0.0012755000990295109, 'cohesion':

In [10]:
for i, j, data in G["graph"][1].edges(data=True) :
    print(f"({i},{j}) => interaction: {functions.getActivity(data['interaction_index'])}")

(0,43) => interaction: to share content, chat in a social network platform
(0,83) => interaction: to study in group or work on collaborative projects
(0,11) => interaction: to be teammates in a sports team
(0,17) => interaction: to share content, chat in a social network platform
(0,21) => interaction: to be volunteers
(1,39) => interaction: to play games
(1,49) => interaction: to be volunteers
(1,45) => interaction: to participate in debate/group presentation & discussion
(2,96) => interaction: to play games
(2,12) => interaction: to be volunteers
(2,36) => interaction: to be teammates in a sports team
(2,54) => interaction: to study in group or work on collaborative projects
(3,82) => interaction: to participate in debate/group presentation & discussion
(3,57) => interaction: to be volunteers
(4,79) => interaction: to share content, chat in a social network platform
(4,36) => interaction: to be teammates in a sports team
(4,58) => interaction: to be teammates in a sports team
(4,77) 